# **DIC interpretation using nonlinear FE model**

This notebook shows how to derive a deeper interpretation of the DIC displacements measured on a grid.
Based on an FE-mesh, it delivers the strain, stress and damage tensors. Further processing of the 
anisotropic damage tensor is used to detect the localized cracks.

In [1]:
%matplotlib widget

In [2]:
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from tvtk.api import tvtk
from mayavi import mlab
from scipy.interpolate import interp2d
from scipy.signal import argrelextrema
from bmcs_shear.api import CrackBridgeAdv
from bmcs_shear.dic_crack import \
    DICGrid, DICAlignedGrid, DICCOR, DICStateFields
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

## Transform material point data of FE mesh to a numpy grid

Rearrange the regular element grid with integration points into a 2D point grid  

In [3]:
def transform_mesh_to_grid(field_Em, n_E, n_F):
    field_Em_shape = field_Em.shape
    # reshape into EFmn and preserve the dimensionality of the input field
    field_EFmn_shape = (n_E, n_F, 2, 2) + field_Em_shape[2:]
    # reorder the Gauss points to comply with the grid point order
    # this reordering might be parameterized by the finite-element formulation
    field_EFmn = field_Em[:, (0, 3, 1, 2)].reshape(*field_EFmn_shape)
    # swap the dimensions of elements and gauss points
    field_EmFn = np.einsum('EFmn...->EmFn...', field_EFmn)
    # merge the element index and gauss point subgrid into globarl point indexes 
    field_MN_shape = (2*n_E, 2*n_F) + field_Em_shape[2:]
    # reshape the field
    field_MN = field_EmFn.reshape(*field_MN_shape)
    return field_MN

## Smoothing of a field

\begin{align}
\bar{f}(x) = \int   \frac{\alpha(x - \xi)}{\int \alpha(x - \xi)\;\mathrm{d}\xi} f(\xi) \; \mathrm{d}\xi
\end{align}

Gigen the coordinate grid $x_{JK}, y_{JK}$ calculate the distances between each two nodes (JK), (MN).
as 
\begin{align}
 r_{IJMN} = (x_{IJ}-x_{MN})^2 + ( y_{IJ} - y_{MN} )^2 
\end{align}

Scale the x-y plane to a unit square

Use the ironing function
\begin{align}
 \alpha = \exp{\left(-\frac{r^2}{2R^2}\right)}
\end{align}
in the integral above

Note that this procedure provides serves for non-local averaging of field function weighted by the distance from an actual point.

In [4]:
def get_z_MN_ironed(x_JK, y_JK, z_JK, RR):
    n_J, n_K = x_JK.shape
    delta_x_JK = x_JK[None, None, ...] - x_JK[..., None, None]
    delta_y_JK = y_JK[None, None, ...] - y_JK[..., None, None]
    r2_n = (delta_x_JK**2 + delta_y_JK**2) / (2*RR**2)
    alpha_r_MNJK = np.exp(-r2_n)
    a_MN = np.trapz(np.trapz(alpha_r_MNJK, x_JK[:, 0], axis=-2), y_JK[0, :], axis=-1)
    normed_a_MNJK = np.einsum('MNJK,MN->MNJK', alpha_r_MNJK, 1 / a_MN)
    z_MNJK = np.einsum('MNJK,JK...->MNJK...', normed_a_MNJK, z_JK)
    # note that the inner integral cancels the dimension J on the axis with 
    # index 2. Therefore, the outer integral integrates over K - again on 
    # the axis with index 2
    z_MN = np.trapz(np.trapz(z_MNJK, x_JK[:, 0], axis=2), y_JK[0, :], axis=2)
    return z_MN

## Interpolate damage field

In [5]:
def interp_omega(x_MN, y_MN, omega_MN, n_x, n_y):
    x_M, x_N = x_MN[:, 0], y_MN[0, :]
    f_omega = interp2d(x_M, x_N, omega_MN.T, kind='cubic')
    xx_M = np.linspace(x_M[-1], x_M[0], n_x)
    yy_N = np.linspace(x_N[0], x_N[-1], n_y)
    xx_NM, yy_NM = np.meshgrid(xx_M, yy_N)
    omega_ipl_NM = f_omega(xx_M, yy_N)
    omega_ipl_NM[omega_ipl_NM < 0] = 0
    return xx_NM, yy_NM, omega_ipl_NM

## Crack tracing algorithm

In [6]:
def crack_detection(xx_MN, yy_MN, omega_MN):
    N_range = np.arange(yy_MN.shape[1])
    omega_NM = omega_MN.T
    n_N, n_M = omega_NM.shape
    omega_NM[omega_NM < 0.15] = 0  # cutoff small damage values
    # smooth the landscape
    # initial crack positions at the bottom of the zone
    arg_C = argrelextrema(omega_NM[0, :], np.greater)[0]
    if len(arg_C) == 0:
        return np.zeros((n_N, 0)), np.zeros((n_N, 0))
    # list of intervals decomposing the crack
    intervals_Cp_ = []
    # distance between right interval boundary and crack position
    arg_C_shift_ = []
    # list of crack horizontal indexes for each horizontal slice
    arg_x_NC_ = [np.copy(arg_C)]
    for N1 in N_range[1:]:
        # horizontal indexes of midpoints between cracks
        arg_C_left_ = np.hstack([[int(arg_C[0]/2)], 
                                 np.array((0.25*arg_C[:-1] + 0.75*arg_C[1:]), 
                                          dtype=np.int_)])
        arg_C_right_ = arg_C + 1
        # array of intervals - first index - crack, second index (left, right)
        intervals_Cp = np.vstack([arg_C_left_, arg_C_right_]).T
        # index distance from the right boundary of the crack interval
        arg_C_shift = np.array([
            np.argmax(omega_NM[N1, interval_p[-1]:interval_p[0]:-1])
            for interval_p in intervals_Cp
        ])
        # cracks, for which the next point could be identified
        C_shift = arg_C_shift > 0
        # next index position of the crack
        arg_C[C_shift] = intervals_Cp[C_shift, -1] - arg_C_shift[C_shift]
        arg_x_NC_.append(np.copy(arg_C))
        # for debugging
        intervals_Cp_.append(intervals_Cp)
        arg_C_shift_.append(arg_C_shift)
    arg_x_NC = np.array(arg_x_NC_)
    n_C = arg_x_NC.shape[1]
    arg_y_C = np.arange(n_N)
    arg_y_NC = np.repeat(arg_y_C, n_C).reshape(n_N, -1)
    xx_NC = xx_MN[arg_x_NC, arg_y_NC]
    yy_NC = yy_MN[arg_x_NC, arg_y_NC]
    return xx_NC, yy_NC

## 3D Plotting functions 

show scalar and tensor field in mayavi

In [39]:
def mlab_tensor(x_NM, y_NM, omega_NM, tensor_MNab, factor=100, label='damage'):
    mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = label
    scene.scene.background = (1.0, 1.0, 1.0)
    scene.scene.foreground = (0.0, 0.0, 0.0)
    scene.scene.z_plus_view()
    scene.scene.parallel_projection = True
    pts_shape = x_NM.shape + (1,)
    pts = np.empty(pts_shape + (3,), dtype=float)
    pts[..., 0] = x_NM[..., np.newaxis]
    pts[..., 1] = y_NM[..., np.newaxis]
    #pts[..., 2] = omega_NM[..., np.newaxis] * factor
    tensor_MNa, _ = np.linalg.eig(tensor_MNab)
    max_tensor_MN = np.max(tensor_MNa, axis=-1)
    max_tensor_NM = max_tensor_MN.T
    max_tensor_NM[max_tensor_NM < 0] = 0
    pts[..., 2] = max_tensor_NM[..., np.newaxis] * factor
    pts = pts.transpose(2, 1, 0, 3).copy()
    pts.shape = int(pts.size / 3), 3
    sg = tvtk.StructuredGrid(dimensions=pts_shape, points=pts)
    sg.point_data.scalars = omega_NM.ravel()
    sg.point_data.scalars.name = 'damage'
    delta_23 = np.array([[1, 0, 0], [0, 1, 0]], dtype=np.float_)
    tensor_MNab_3D = np.einsum('...ab,ac,bd->...cd', tensor_MNab, delta_23, delta_23)
    sg.point_data.tensors = tensor_MNab_3D.reshape(-1,9)
    sg.point_data.tensors.name = label
    # Now visualize the data.
    d = mlab.pipeline.add_dataset(sg)
    mlab.pipeline.iso_surface(d)
    mlab.pipeline.surface(d)
    mlab.show()

In [8]:
def mlab_scalar(x_NM, y_NM, z_NM, factor=100, label='damage'):
    mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = label
    scene.scene.background = (1.0, 1.0, 1.0)
    scene.scene.foreground = (0.0, 0.0, 0.0)
    scene.scene.z_plus_view()
    scene.scene.parallel_projection = True
    pts_shape = xx_NM.shape + (1,)
    pts = np.empty(pts_shape + (3,), dtype=float)
    pts[..., 0] = x_NM[..., np.newaxis]
    pts[..., 1] = y_NM[..., np.newaxis]
    pts[..., 2] = z_NM[..., np.newaxis] * factor
    pts = pts.transpose(2, 1, 0, 3).copy()
    pts.shape = int(pts.size / 3), 3
    sg = tvtk.StructuredGrid(dimensions=pts_shape, points=pts)
    sg.point_data.scalars = z_NM.T.ravel()
    sg.point_data.scalars.name = label
    d = mlab.pipeline.add_dataset(sg)
    mlab.pipeline.iso_surface(d)
    mlab.pipeline.surface(d)
    mlab.show()

# DIC grid

## Input displacements on a grid

In [10]:
dic_grid = DICGrid(start_t=0, end_t=1, U_factor=100, dir_name='B1_TV1')
dic_grid.sz_bd.Rectangle = True
dic_grid.sz_bd.matrix_.trait_set(f_t=0.3 * 38**(2/3), f_c=38, d_a=16, E_c=28000)
dic_grid.sz_bd.trait_set(H=350, B=320, L=1500)
dic_grid.sz_bd.csl.add_layer(CrackBridgeAdv(z=50, n=5, d_s=16, E=210000, f_c=38))

## Finite element processing of the displacement field

In [12]:
dsf = DICStateFields(dic_grid=dic_grid)
dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.01);
dsf.eval()

# State fields on a regular grid

In [24]:
t_idx = -5
kappa_Emr = dsf.hist.state_vars[t_idx][0]['kappa']
omega_Emr = dsf.hist.state_vars[t_idx][0]['omega']
phi_Emab = dsf.tmodel_._get_phi_Emab(kappa_Emr)

In [25]:
n_E, n_F = dic_grid.n_x-1, dic_grid.n_y-1

## Spatial coordinates

In [26]:
x_MNa = transform_mesh_to_grid(dsf.xmodel.x_Ema, n_E, n_F)
x_aMN = np.einsum('MNa->aMN', x_MNa)
x_MN, y_MN = x_aMN

In [27]:
max_sig = 5
max_eps = 0.02

## Strain tensor field

In [28]:
U_o = dsf.hist.U_t[t_idx]
eps_Emab = dsf.xmodel.map_U_to_field(U_o)
eps_MNab = transform_mesh_to_grid(eps_Emab, n_E, n_F)
eps_MNa, _ = np.linalg.eig(eps_MNab)
max_eps_MN = np.max(eps_MNa, axis=-1)
max_eps_MN[max_eps_MN < 0] = 0

## Stress tensor field

In [29]:
sig_Emab, _ = dsf.tmodel_.get_corr_pred(eps_Emab, 1, kappa_Emr, omega_Emr)
sig_MNab = transform_mesh_to_grid(sig_Emab, n_E, n_F)
sig_MNa, _ = np.linalg.eig(sig_MNab)
max_sig_MN = np.max(sig_MNa, axis=-1)
max_sig_MN[max_sig_MN < 0] = 0

## Integrity tensor field

In [30]:
phi_MNab = transform_mesh_to_grid(phi_Emab, n_E, n_F)
omega_MNab = np.identity(2) - phi_MNab
phi_MNa, _ = np.linalg.eig(phi_MNab)
min_phi_MN = np.min(phi_MNa, axis=-1)
omega_MN = 1 - min_phi_MN
omega_MN[omega_MN < 0.2] = 0

## Cumulative analysis of stress-strain in the field

In [31]:
kappa_zero = np.zeros_like(kappa_Emr[0,0,:])
omega_zero = np.zeros_like(kappa_zero)
eps_test = np.zeros((2,2), dtype=np.float_)
eps_range = np.linspace(0, 0.5, 1000)
sig_range = []
for eps_i in eps_range:
    eps_test[0, 0] = eps_i
    arg_sig, _ = dsf.tmodel_.get_corr_pred(eps_test, 1, kappa_zero, omega_zero)
    sig_range.append(arg_sig)
#max_eps = np.max(max_eps_MN)
arg_max_eps = np.argwhere(eps_range > max_eps)[0][0]
sig_range = np.array(sig_range, dtype=np.float_)
G_f = np.trapz(sig_range[:, 0, 0], eps_range)

## Crack detection

In [32]:
xx_NM, yy_NM, omega_ipl_NM = interp_omega(x_MN, y_MN, omega_MN, 116, 28)
xx_MN, yy_MN, omega_ipl_MN = xx_NM.T, yy_NM.T, omega_ipl_NM.T
omega_irn_MN = get_z_MN_ironed(xx_MN, yy_MN, omega_ipl_MN, 8)
xx_NC, yy_NC = crack_detection(xx_MN, yy_MN, omega_irn_MN)

# State fields - 2D Plots 

In [33]:
fig, ((ax_eps, ax_FU), (ax_sig, ax_sig_eps), (ax_omega, ax_cracks)) = plt.subplots(
    3, 2, figsize=(14,7), tight_layout=True)
cs_eps = ax_eps.contourf(x_aMN[0], x_aMN[1], max_eps_MN, cmap='BuPu', 
                         vmin=0, vmax=max_eps)
cbar_eps = fig.colorbar(cm.ScalarMappable(norm=cs_eps.norm, cmap=cs_eps.cmap), 
                        ax=ax_eps, ticks=np.arange(0, max_eps*1.01, 0.005),
                        orientation='horizontal')
cbar_eps.set_label(r'$\max(\varepsilon_I) > 0$')
ax_eps.axis('equal')
ax_eps.axis('off')
cs_sig = ax_sig.contourf(x_aMN[0], x_aMN[1], max_sig_MN, cmap='Reds',
                         vmin=0, vmax=max_sig)
cbar_sig = fig.colorbar(cm.ScalarMappable(norm=cs_sig.norm, cmap=cs_sig.cmap), 
                        ax=ax_sig, ticks=np.arange(0, max_sig*1.01, 0.5),
                        orientation='horizontal')
cbar_sig.set_label(r'$\max(\sigma_I) > 0$')
ax_sig.axis('equal')
ax_sig.axis('off')
cs = ax_omega.contourf(x_aMN[0], x_aMN[1], omega_MN, cmap='BuPu', vmin=0, vmax=1)
cbar_omega = fig.colorbar(cm.ScalarMappable(norm=cs.norm, cmap=cs.cmap), 
                          ax=ax_omega, ticks=np.arange(0, 1.1, 0.2),
                          orientation='horizontal')
cbar_omega.set_label(r'$\omega = 1 - \min(\phi_I)$')
ax_omega.axis('equal');
ax_omega.axis('off')

dsf.dic_grid.plot_load_deflection(ax_FU)

ax_sig_eps.plot(eps_MNa[..., 0].flatten(), sig_MNa[..., 0].flatten(), 'o', color='green')
ax_sig_eps.plot(eps_range[:arg_max_eps], sig_range[:arg_max_eps, 0, 0], 
                color='white', lw=2, label='$G_f$ = %g [N/mm]' % G_f)
ax_sig_eps.set_xlabel(r'$\varepsilon$ [-]')
ax_sig_eps.set_ylabel(r'$\sigma$ [MPa]')
ax_sig_eps.legend()

ax_cracks.plot(xx_NC, yy_NC, color='black', linewidth=3);
ax_cracks.contour(xx_MN, yy_MN, omega_irn_MN, cmap=cm.coolwarm, antialiased=False)
ax_cracks.axis('equal')
ax_cracks.axis('off');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 3D plots 

In [41]:
eps_irn_MNab = get_z_MN_ironed(x_MN, y_MN, eps_MNab, 8)

In [42]:
mlab_tensor(x_MN.T, y_MN.T, omega_MN.T, eps_irn_MNab, factor=5000)
# mlab_scalar(xx_NM, yy_NM, omega_ipl_NM)
# mlab_scalar(xx_NM, yy_NM, omega_irn_MN.T)